In [1]:
import requests


In [2]:
resp = requests.get("https://api.openalex.org/topics")
resp

<Response [200]>

In [3]:
resp.json()

{'meta': {'count': 4516,
  'db_response_time_ms': 6,
  'page': 1,
  'per_page': 25,
  'groups_count': None},
 'results': [{'id': 'https://openalex.org/T11881',
   'display_name': 'Crystallization Processes and Control',
   'description': 'This cluster of papers focuses on the crystallization processes and control, including topics such as nucleation, solubility, polymorphism, ultrasound-assisted crystallization, process analytical technology, crystal growth, pharmaceutical crystallization, continuous crystallization, and crystal engineering.',
   'keywords': ['Crystallization',
    'Nucleation',
    'Solubility',
    'Polymorphism',
    'Ultrasound-Assisted Crystallization',
    'Process Analytical Technology',
    'Crystal Growth',
    'Pharmaceutical Crystallization',
    'Continuous Crystallization',
    'Crystal Engineering'],
   'ids': {'openalex': 'https://openalex.org/T11881',
    'wikipedia': 'https://en.wikipedia.org/wiki/Crystallization'},
   'subfield': {'id': 'https://opena

In [10]:
import requests

# API endpoint and topic ID
url = "https://api.openalex.org/works"
params = {
    "filter": f"topics.id:https://openalex.org/T14271",
    "page": 1,
    "per-page": 20
}
headers = {"accept": "application/json"}

# Make the API request
response = requests.get(url, params=params, headers=headers)
works_data = response.json()

In [13]:
for work in works_data["results"]:
    print(work["title"])
    print("---")

Human Motives and Cultural Models
---
Preference for delayed reinforcement: An experimental study of a cultural observation.
---
Contribución al conocimiento de los centros nerviosos de los insectos
---
La educación superior en tiempos de pandemia: una visión desde dentro del proceso formativo
---
Behavioral Psychology/Psicología Conductual
---
Revista Colombiana de Psicología
---
Corporal Expression as a broad spectrum psycho-pedagogical resource
---
Psicología del desarrollo moral
---
Motivation to learn
---
PRECISION TEACHING: DISCOVERIES AND EFFECTS
---
Emotional intelligence and social and academic adaptation to school.
---
Abstract concept learning in the pigeon.
---
Institutional Neurosis
---
Gender differences in emotional intelligence: The mediating effect of age.
---
NEUROSIS AND THE MEXICAN FAMILY STRUCTURE
---
¿Es la inteligencia emocional una cuestión de género? Socialización de las competencias emocionales en hombres y mujeres y sus implicaciones
---
The effect of verbal 